In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from labels_condensatore import labels_condensatore, dict_labels

In [ ]:
import scipy.io
array_dict = {'5V':{},
              '12V':{},
              '24V': {}}
for v in ['12']:
  for name in ['RE95']:
    filename = f'{name}_{v}V'
    mat = scipy.io.loadmat(f'{filename}.mat')
    arr = mat[filename]

    colonne_da_estrarre = [1, 2]
    result = [[row[i] for i in colonne_da_estrarre] for row in arr]

    array_dict[f'{v}V'][name] = result
    print(name)
    print(mat[filename])

In [ ]:
import zipfile

# Percorso del file zip da decomprimere
percorso_file_zip = 'Dati_Dataset_conduttivi.zip'

# Percorso della cartella di destinazione per l'estrazione
percorso_cartella_destinazione = './'

# Decomprimi il file zip
with zipfile.ZipFile(percorso_file_zip, 'r') as zip_ref:
    zip_ref.extractall(percorso_cartella_destinazione)

In [ ]:
def is_within_range(value, range_tuple, type):

    lower_bound, upper_bound = range_tuple
    if type == 'd':
      return 10 ** lower_bound <= value <= 10 ** upper_bound
    else:
      return lower_bound <= value <= upper_bound

def assign_label(v, name, ex):
  D, gamma = labels_condensatore()['test_conductive'][f'{v}V'][name][ex]

  for item in dict_labels().items():
    label, values = item
    range_g,range_d = values['g'], values['d']

    if is_within_range(gamma, range_g, 'g') and is_within_range(D, range_d, 'd'):
      return label

  return -1


In [ ]:
#voltage = [5,10,12,15,20]
voltage = ['0.5','1','2','3','5']
experiments = [x+1 for x in range(10)]
names = ['aria_aria','alcool_iso', 'deion', 'rubin', 'ultra', 'novec', 'sf33']
#names = ['aria', 'etanolo','RE1','RE5','RE10', 'RE25', 'RE75','RE95', 'RE99', 'RE90']
#names = ['deion', 'rubin']
dataset = None;


In [ ]:
for name in names:
  path_base = 'Dati_Dataset_conduttivi/' + name
  for v in voltage:

    dati_tmp = pd.read_csv(f'{path_base}/{v}V_1.CSV', skiprows=25,header=None).iloc[:, 1]

    label_tmp = assign_label(v, name, 0)
    dati_tmp.loc[10001] = label_tmp
    for e in experiments[1:]:
      if name == 'RE90' and int(v) == 24 and e == 10:
        break

      if name == 'RE95' and (int(v) == 12 or int(v) == 24) and e == 10:
        break

      dati_tmp_2 = pd.read_csv(f'{path_base}/{v}V_{e}.CSV', skiprows=25,header=None).iloc[:, 1]
      label_tmp_2 = assign_label(v, name, e-1)
      dati_tmp_2.loc[10001] = label_tmp_2
      dati_tmp = pd.concat([dati_tmp,dati_tmp_2], axis = 1)

    if dataset is None:
      dataset = dati_tmp
    else:
      dataset = pd.concat([dataset,dati_tmp], axis = 1)

In [ ]:
data = dataset.transpose()
data = data.values
np.save('Dataset_Condensatore_Conduttivi.npy', data)

In [ ]:
X = data[:, :-1]
y = data[:, -1]